In [ ]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
from skeleton import *
from sklearn.metrics import (recall_score, f1_score,
                             precision_score, accuracy_score)
import os

In [ ]:
def testClassifier(clf,train,y_train,test,y_test):
    print("=== ",clf.__class__.__name__,end='')
    print(":",vars(clf))
    clf=clf.fit(train,y_train)
    #Predecimos las etiqueta del conjunto de prueba
    yp=clf.predict(test)
    # Medidas de bondad
    scores={}

    scores['Accuracy']=accuracy_score(y_test,yp)
    scores['Macro F1']=f1_score(y_test,yp,average='macro')
    scores['Macro Recall']=recall_score(y_test,yp,average='macro')
    scores['F1 is_humor=1']=f1_score(y_test,yp)
    
    print("Accuracy","Macro F1","Macro Recall","F1 is_humor=1",sep='\t')
    print("{:.6f}\t{:.6f}\t{:.6f}\t{:.6f}".format(
        scores['Accuracy'],scores['Macro F1'],scores['Macro Recall'],scores['F1 is_humor=1']))
                            
#     print("Recall: ", scores['recall'])
#     print("F1: ", scores['f1'])
#     print("Accuracy: ", scores['accuracy'])
#     print("F1 Humor: ", scores['f1_humor'])
    #Vemos la frontera de decisión
    plotDecisionBoundary(clf,train, y_train)
    return scores

## Un problema de texto  (Identificación de humor HAHA -2019)

Para este ejemplo utilizaremos los datos proporcionados para la tarea 1 del HAHA - Humor Analysis based on Human Annotation, la cual consiste determinar si tweets en español son humorísticos o no (un problema de clasificación binaria). Esta tarea forma parte del Iberian Languages Evaluation Forum (IberLEF 2019).

### Corpus 

El conjunto de datos fue etiquetado como se indica [5]. Todos los tweets están etiquetados como humorístico **is_humor=1**  o no humorístico **is_humor=0**. Los datos relevantes para el problema se clasificación ser muestran en el siguiente ejemplo:

|Desscripción|Clave|Valor|
|:------:|:------:|:-----:|
|Tweet |text|Después de la tormenta sale... Tu mamá gritando porque no metiste la ropa.|
|Etiqueta de clase|is_humor|1|
|Representación vectorial|vec| Vectores de dimensión 300 (FastText preentrenados para español) |
|Identificador|id|942079817905770496|



In [ ]:
# Cargamos lod datos.
train_data=pd.read_json('data/haha_train_ft_pre.json',lines=True)

In [ ]:
#Vemos los primeros 5
train_data.head()[['id','text','is_humor','vec']]

In [ ]:
#Dimensión de los vectores
len(train_data.vec[3])

In [ ]:
# Los datos de prueba
test_data=pd.read_json('data/haha_test_ft_pre.json',lines=True)

In [ ]:
#Obtenemos la representación vectorial para los conjunto de prueba y entrenamiento
train,y_train=np.array([np.array(x) for x in train_data.vec]),np.array(train_data.is_humor)
test,y_test=np.array([np.array(x) for x in test_data.vec]),np.array(test_data.is_humor)

In [ ]:
P={"FastText":{"Accuracy":0.809050,"Macro F1":0.793270,"Macro Recall":0.786900,"F1 is_humor=1":0.736140}} #Performance

# 
P["NC.cos.avg"]=testClassifier(NearestCentroid(distance='coseno',centroid_type='Average'),train,y_train,test,y_test)
P["NC.euc.avg"]=testClassifier(NearestCentroid(distance='euclidiana',centroid_type='Average'),train,y_train,test,y_test)

P["NC.cos.sum"]=testClassifier(NearestCentroid(distance='coseno',centroid_type='Sum'),train,y_train,test,y_test)
P["NC.euc.sum"]=testClassifier(NearestCentroid(distance='euclidiana',centroid_type='Sum'),train,y_train,test,y_test)

P["NC.cos.rocchio"]=testClassifier(NearestCentroid(distance='coseno',centroid_type='Rocchio',beta=16,gamma=4),train,y_train,test,y_test)
P["NC.euc.rocchio"]=testClassifier(NearestCentroid(distance='euclidiana',centroid_type='Rocchio',beta=16,gamma=4),train,y_train,test,y_test)

P["NC.cos.nsum"]=testClassifier(NearestCentroid(distance='coseno',centroid_type='NormSum'),train,y_train,test,y_test)
P["NC.euc.nsum"]=testClassifier(NearestCentroid(distance='euclidiana',centroid_type='NormSum'),train,y_train,test,y_test)


P["KNN.cos.u"]=testClassifier(kNN(distance='coseno',k=5,weight_type='uniform'),train,y_train,test,y_test)
P["KNN.euc.u"]=testClassifier(kNN(distance='euclidiana',k=5,weight_type='uniform'),train,y_train,test,y_test)

P["KNN.cos.m"]=testClassifier(kNN(distance='coseno',k=5,weight_type='mean_dist'),train,y_train,test,y_test)
P["KNN.euc.m"]=testClassifier(kNN(distance='euclidiana',k=5,weight_type='mean_dist'),train,y_train,test,y_test)

P["KNN.cos.w"]=testClassifier(kNN(distance='coseno',k=5,weight_type='weighed_dist'),train,y_train,test,y_test)
P["KNN.euc.w"]=testClassifier(kNN(distance='euclidiana',k=5,weight_type='weighed_dist'),train,y_train,test,y_test)

In [ ]:
from toolbox import *

In [5]:
!wc -l data/haha_*.json

     240 data/haha_all_ft_pre_min10.json
    7201 data/haha_test_ft_pre.json
      72 data/haha_test_ft_pre_min10.json
   16799 data/haha_train_ft_pre.json
     168 data/haha_train_ft_pre_min10.json
   24480 total


In [17]:
!wc -l data/haha_train_ft_pre.json | cut -d' ' -f1

16799


In [18]:
import subprocess

result = subprocess.check_output("$(wc -l data/haha_train_ft_pre.json | cut -d' ' -f1)", shell=True)
result

CalledProcessError: Command 'wc -l data/haha_train_ft_pre.json | cut -d -f1' returned non-zero exit status 1.

In [10]:
train_n=os.system('wc -l haha_train_ft_pre.json')
print(sys.stdout)
test_n=os.system('wc -l haha_test_ft_pre.json')
train_n,test_n

(256, 256)

In [ ]:
!wc -l data/haha_*.json
# !cat data/haha_train_ft_pre_min10.json > data/haha_all_ft_pre_min10.json
# !cat data/haha_test_ft_pre_min10.json >> data/haha_all_ft_pre_min10.json
# !wc -l data/haha_all_ft_pre_min10.json
!cat data/haha_train_ft_pre.json > data/haha_all_ft_pre.json
!cat data/haha_test_ft_pre.json >> data/haha_all_ft_pre.json
!wc -l data/haha_all_ft_pre.json

!rm -f *.pickle
inv_idx = InvertedIdx("data/haha_all_ft_pre.json")
inv_idx.process(showProgressEach=10000, stemm=False)
inv_idx.compute_mtx()
tfidf=pd.DataFrame(inv_idx.idx_mtx.todense(), index=inv_idx.corpus).T
tfidf

In [ ]:
train2=np.ascontiguousarray(tfidf.head(168).to_numpy())
test2=np.ascontiguousarray(tfidf.tail(72).to_numpy())
train2.shape,test2.shape

In [ ]:
P2={} #Performance

# 
P2["NC.cos.avg"]=testClassifier(NearestCentroid(distance='coseno',centroid_type='Average'),train2,y_train,test2,y_test)
P2["NC.euc.avg"]=testClassifier(NearestCentroid(distance='euclidiana',centroid_type='Average'),train2,y_train,test2,y_test)

P2["NC.cos.sum"]=testClassifier(NearestCentroid(distance='coseno',centroid_type='Sum'),train2,y_train,test2,y_test)
P2["NC.euc.sum"]=testClassifier(NearestCentroid(distance='euclidiana',centroid_type='Sum'),train2,y_train,test2,y_test)

P2["NC.cos.rocchio"]=testClassifier(NearestCentroid(distance='coseno',centroid_type='Rocchio',beta=16,gamma=4),train2,y_train,test2,y_test)
P2["NC.euc.rocchio"]=testClassifier(NearestCentroid(distance='euclidiana',centroid_type='Rocchio',beta=16,gamma=4),train2,y_train,test2,y_test)

P2["NC.cos.nsum"]=testClassifier(NearestCentroid(distance='coseno',centroid_type='NormSum'),train2,y_train,test2,y_test)
P2["NC.euc.nsum"]=testClassifier(NearestCentroid(distance='euclidiana',centroid_type='NormSum'),train2,y_train,test2,y_test)


P2["KNN.cos.u"]=testClassifier(kNN(distance='coseno',k=5,weight_type='uniform'),train2,y_train,test2,y_test)
P2["KNN.euc.u"]=testClassifier(kNN(distance='euclidiana',k=5,weight_type='uniform'),train2,y_train,test2,y_test)

P2["KNN.cos.m"]=testClassifier(kNN(distance='coseno',k=5,weight_type='mean_dist'),train2,y_train,test2,y_test)
P2["KNN.euc.m"]=testClassifier(kNN(distance='euclidiana',k=5,weight_type='mean_dist'),train2,y_train,test2,y_test)

P2["KNN.cos.w"]=testClassifier(kNN(distance='coseno',k=5,weight_type='weighed_dist'),train2,y_train,test2,y_test)
P2["KNN.euc.w"]=testClassifier(kNN(distance='euclidiana',k=5,weight_type='weighed_dist'),train2,y_train,test2,y_test)

In [ ]:
df=pd.DataFrame(P).T.sort_values("F1 is_humor=1",ascending =False)
df

In [ ]:
df2=pd.DataFrame(P2).T.sort_values("F1 is_humor=1",ascending =False)
df2

In [ ]:
df.describe()

In [ ]:
df2.describe()